In [1]:
import json

In [2]:
ds_all_json_path = '../Datasets/UIHNJMuLv3/cls_folds/train_validation_test_fold.json'

In [3]:
with open(ds_all_json_path, 'r') as f:
    ds_all = json.load(f)

In [4]:
ds_all.keys()

dict_keys(['code', 'idmap', 'train_statistics', 'validation_statistics', 'test_statistics', 'train', 'validation', 'test'])

In [5]:
jobs_json_dict = {"position": {'code':['inside','outside']},
                  "quality":{'code':['fine','nonsense']}, 
                  "ileocecal":{'code':['nofeature','ileocecal']}, 
                  "cleansing":{'code':['bbps0', 'bbps1', 'bbps2', 'bbps3']}}

In [6]:
ds_all['code']

['outside', 'nonsense', 'ileocecal', 'bbps0', 'bbps1', 'bbps2', 'bbps3']

In [7]:
ds_all.keys

<function dict.keys>

In [8]:
# labels的顺序是['outside', 'nonsense', 'ileocecal', 'bbps0', 'bbps1', 'bbps2', 'bbps3']
# 每个位置对应一个float值，代表了这个位置的概率
def check_and_pick_for_job(job_name, sample:list):
    job_file = sample[0]
    sample_labels = sample[1][1]
    if job_name == "position":
        job_label = [sample_labels[0]]
        return job_file, job_label
    elif job_name == "quality": # quality 要求outside必须为0
        if sample_labels[0] == 1:
            return None, None
        else:
            job_label = [sample_labels[1]]
            return job_file, job_label
    elif job_name == "ileocecal": # ileocecal 要求outside和nonsense必须为0
        if sample_labels[0] == 1:
            return None, None
        elif sample_labels[1] == 1:
            return None, None
        else:
            job_label = [sample_labels[2]]
            return job_file, job_label
    elif job_name == "cleansing": # cleansing 要求outside和nonsense必须为0
        if sample_labels[0] == 1:
            return None, None
        elif sample_labels[1] == 1:
            return None, None
        else:
            job_label = sample_labels[3:]
            return job_file, job_label
    else:
        raise ValueError("job_name must be one of ['position', 'quality', 'ileocecal', 'cleansing']")
       

In [9]:
from collections import OrderedDict

for job_name in jobs_json_dict.keys():
    for mode in ['train', 'validation', 'test']:
        jobs_json_dict[job_name][mode] = OrderedDict()
        for set_name in ds_all[mode].keys():
            data_origin = ds_all[mode][set_name]
            for entry in data_origin.items():
                job_file, job_label = check_and_pick_for_job(job_name, entry)
                if job_file is None:
                    continue
                else:
                    jobs_json_dict[job_name][mode][job_file] = job_label
            

In [10]:
for job_name in jobs_json_dict.keys():
        json.dump(jobs_json_dict[job_name], open(f'../Datasets/UIHNJMuLv3/cls_folds/job_{job_name}_train_validation_test_fold.json', 'w'), indent=4)

In [12]:
import torch
import torch.nn.functional as F
# loss function
import torch.nn as nn


In [ ]:
F.cross_entropy(torch.tensor([[0.1, 0.2, 0.3, 0.4], [0.1, 0.2, 0.3, 0.4]]), torch.tensor([0, 1]))